# Instructor Do: Plaid Demo

In [1]:
# Initial imports
import plaid
import datetime
import json
import os
from dotenv import load_dotenv

### Set environment variables from .env

In [2]:
load_dotenv()

True

### Retrieve env variables

In [3]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")
PLAID_SBX_SECRET_KEY = os.getenv("PLAID_SBX_SECRET_KEY")

### Create Client Object

In [4]:
# Create client object
client = plaid.Client(
    client_id=PLAID_CLIENT_ID,
    secret=PLAID_SBX_SECRET_KEY,
    public_key=PLAID_PUBLIC_KEY,
    environment="sandbox"
)

### Fetch List of Financial Institutions

In [5]:
# Get institutions
client.Institutions.get(1)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []}],
 'request_id': '2ICmtzIpGgOHJiL',
 'total': 11425}

### Create Public Token and Exchange for Access Token

In [6]:
# Select an institution for processing
INSTITUTION_ID = "ins_112060"

# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(
    INSTITUTION_ID,
    ["transactions","income","assets"]
)

# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response["public_token"])

# Store access token as variable
access_token = exchange_response["access_token"]

### Get List of Accounts

In [7]:
# Get accounts associated with institution
client.Accounts.get(access_token)

{'accounts': [{'account_id': 'n7PvWn75yKHb4WVwPQnxCeBa6B3gV6c6WLNlW',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'bKlZ7WKxL1C1AW6DlrwaIeDq5DQJE5cVZM1bq',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'mjNvEnj5mqhbaV3gN4PBCm5do5XxpocLdVnyX',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

### Get Transactions for Accounts

In [8]:
# Get transactions for institution for specific date range
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-30))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())

# Get transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print JSON output
print(json.dumps(transaction_response["transactions"][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "n7PvWn75yKHb4WVwPQnxCeBa6B3gV6c6WLNlW",
        "account_owner": null,
        "amount": 6.33,
        "authorized_date": null,
        "category": [
            "Travel",
            "Taxi"
        ],
        "category_id": "22016000",
        "date": "2020-04-28",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "Uber",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
        "pending_transac